In [ ]:
pip install --upgrade --force-reinstall "numpy<2"

import torch
from diffusers import StableVideoDiffusionPipeline
from diffusers.utils import load_image, export_to_video
import requests
from io import BytesIO
import numpy as np
from PIL import Image
import cv2

pipe = StableVideoDiffusionPipeline.from_pretrained(
    "stabilityai/stable-video-diffusion-img2vid-xt",
    torch_dtype=torch.float32  # Use float32 since float16 requires GPU
).to("cuda")

pipe.enable_model_cpu_offload()

# Import the initial image crawling online
image_url =  "https://t4.ftcdn.net/jpg/01/14/88/65/360_F_114886567_3hT4lx1EAXR4IlIWSakzqT3pWf2Pvoko.jpg"

response = requests.get(image_url)

# Prepare the init image
init_image = Image.open(BytesIO(response.content)).convert("RGB")
init_image = init_image.resize((426, 240))

# Generate first 40 frames video base on the image above
video_frames = pipe(init_image, num_frames = 40).frames

for i, frame in enumerate(video_frames):
    frame.save(f"frame_{i:03d}.png")

# continue for next 40 frames video
next_video_img = video_frames[-1]

video_frames_next = pipe(next_video_img, num_frames = 40).frames

for i, frame in enumerate(video_frames_next):
    frame.save(f"frame_{i+40:03d}.png")

# Setup for the final video
fps = 10

frame = cv2.imread("frame_000.png")

h, w, _ = first_frame.shape

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
video_writer = cv2.VideoWriter("final.mp4", fourcc, fps, (w,h))

#Merge these two video into 80 frames video
for i in range(80):
    frame_path = f"frame_{i:03d}.png"
    frame = cv2.imread(frame_path)
    video_writer.write(frame)

# Release the video
video_writer.release()